<a href="https://colab.research.google.com/github/gott51010/zhenXiang/blob/master/%E5%88%86%E6%89%B9%E6%95%B0%E6%8D%AE%E9%A2%84%E5%A4%84%E7%90%86%E6%A1%88%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
!pip install xfeat

     |████████████████████████████████| 296kB 14.0MB/s 
     |████████████████████████████████| 1.1MB 18.5MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 481kB 56.8MB/s 
     |████████████████████████████████| 143kB 42.3MB/s 
     |████████████████████████████████| 112kB 35.8MB/s 
     |████████████████████████████████| 51kB 4.2MB/s 
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-cp36-none-any.whl size=7849 sha256=0847cafce34910f4973d1c359c1e88f1dd29600c621111aa9a780ed388b99fb9
  Stored in directory: /root/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
  Created wheel for alembic: filename=alembic-1.5.3-py2.py3-none-any.whl size=155547 sha256=80c338bd2db6948b41bfe44f920d879c2325d3661b283abf6127ade0992267c0
  Stored in directory: /root/.cache/pip/wheels/7f/ed/1b/a66a0cbca75fd3e374bd5cc60c443e5675f10ef1f1f78ec31d
  Created wheel for Mako: filename=Mako-1.1.4-py2.py3-none-any.whl size=75675 sha25

In [71]:
import re
from functools import partial
from glob import glob

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import (ArithmeticCombinations, ConcatCombination,
                   GBDTFeatureExplorer, GBDTFeatureSelector, LabelEncoder,
                   LambdaEncoder, Pipeline, SelectCategorical, SelectNumerical,
                   TargetEncoder, aggregation)
# sns.set(font='IPAGothic')
# matplotlib.rcParams['font.family'] = 'IPAGothic'
# matplotlib.rcParams['font.sans-serif'] = ['IPAGothic','IPAMincho', 'SimHei', 'msyh']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
# matplotlib.rcParams['font.size'] = 12  # 字体大小
# matplotlib.rcParams['axes.unicode_minus'] = False  # 正常显示负号
# plt.rcParams['font.sans-serif']=['IPAGothic'] #显示中文标签
# plt.rcParams['axes.unicode_minus']=False  

In [72]:
sns.set()

In [73]:
# データ前処理用関数を作成

def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(moyori)
    return moyori

In [74]:
def normalize_area(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    return area


In [75]:
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki


In [76]:
paths = glob(r'../content/sample_data/NEW/*')
train_dfs = []

In [77]:
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)
test_df = pd.read_csv(r'../content/sample_data/test.csv')

In [78]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
rm_cols = []

test_df[TARGET] = np.nan
df = pd.concat([train_df, test_df])

rm_cols += ['市区町村コード']

for i, v in df.nunique().iteritems():
    if v <= 1:
        rm_cols.append(i)

train_df.drop(rm_cols, axis=1, inplace=True)
test_df.drop(rm_cols, axis=1, inplace=True)
df = pd.concat([train_df, test_df])

# 特徴量生成

enc_dic = {}
for i, e in enumerate(sorted(list(set(df['取引時点'].values)))):
    enc_dic[e] = i
df['取引時点_enc'] = df['取引時点'].map(enc_dic)

# Target Encoding
# ある行の特徴量として平均値を計算するのに、その時点で過去に登場したデータの集計を用いる
# 以下例では、都道府県ごとに各取引時点より過去の値の平均値を用いる
te_dic = {}
time_col = '取引時点_enc'
group_col = '都道府県名'

for i in set(df[time_col].values):
    tmp_df = df[df[time_col] < i]
    te_dic[i] = tmp_df.groupby(group_col)[TARGET].agg('mean').to_dict()


def calc_te(row):
    if row[time_col] in te_dic and row[group_col] in te_dic[row[time_col]]:
        return te_dic[row[time_col]][row[group_col]]
    else:
        return 0


df[group_col+'_te'] = df.apply(calc_te, axis=1)

df['取引時点_何年前'] = df['取引時点'].apply(lambda x: 2020-int(x[:4]))
df.drop(['取引時点'], axis=1, inplace=True)
df['建築年'] = df['建築年'].apply(lambda x: convert_wareki_to_seireki(x))
df['面積（㎡）'] = df['面積（㎡）'].apply(lambda x: normalize_area(x))
df['最寄駅：距離（分）'] = df['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

# 数値データを抽出しておく
num_df = SelectNumerical().fit_transform(df)

# カテゴリカルデータを抽出
encoder = Pipeline([
    SelectCategorical(),
    LabelEncoder(output_suffix=""),
])

le_df = encoder.fit_transform(df)

# 数値データを組み合わせた特徴量生成
encoder = Pipeline(
    [
        SelectNumerical(),
        ArithmeticCombinations(
            input_cols=["面積（㎡）", "容積率（％）"],
            drop_origin=True,
            operator="*",
            r=2,
        ),
    ]
)

In [79]:
num_comb_df = encoder.fit_transform(df)/100

In [80]:
# 集約特徴量生成
agg_dfs = []


def get_agg_df(df, group_col):

    agg_df, agg_cols = aggregation(df,
                                   group_key=group_col,
                                   # '前面道路：幅員（ｍ）',
                                   group_values=['最寄駅：距離（分）',
                                                 '面積（㎡）', '建ぺい率（％）', '容積率（％）'],
                                   agg_methods=['count', 'mean', 'min', 'max'],
                                   )

    return agg_df[agg_cols]

In [81]:
group_col = '市区町村名'
agg_dfs.append(get_agg_df(df, group_col))

# 生成した特徴量を結合
feat_df = pd.concat([num_df, le_df, num_comb_df]+agg_dfs, axis=1)

# モデル構築
train_df = feat_df[feat_df['取引時点_何年前'] > 1]
val_df = feat_df[feat_df['取引時点_何年前'] == 1]
test_df = feat_df[feat_df['取引時点_何年前'] == 0]

feat_cols = [col for col in train_df.columns if col not in rm_cols+[ID, TARGET]]

cat_cols = list(le_df.columns) + ['取引時点_enc']

train_x = train_df[feat_cols]
train_y = train_df[TARGET]
val_x = val_df[feat_cols]
val_y = val_df[TARGET]
test_x = test_df[feat_cols]
test_y = test_df[TARGET]

In [82]:
SEED = 0

params = {
    'objective': 'regression',
    'metric': 'mae',
    'num_leaves': 42,
    'max_depth': 7,
    "feature_fraction": 0.8,
    'subsample_freq': 1,
    "bagging_fraction": 0.95,
    'min_data_in_leaf': 2,
    'learning_rate': 0.1,
    "boosting": "gbdt",
    "lambda_l1": 0.1,
    "lambda_l2": 10,
    "verbosity": -1,
    "random_state": 42,
    "num_boost_round": 50000,
    "early_stopping_rounds": 100
}


In [83]:
train_data = lgb.Dataset(train_x, label=train_y)
val_data = lgb.Dataset(val_x, label=val_y)

In [90]:
train_x.head(30)


,最寄駅：距離（分）,面積（㎡）,建築年,建ぺい率（％）,容積率（％）,取引時点_enc,都道府県名_te,取引時点_何年前,都道府県名,市区町村名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装,取引の事情等,面積（㎡）容積率（％）_combi,agg_count_最寄駅：距離（分）_grpby_市区町村名,agg_count_面積（㎡）_grpby_市区町村名,agg_count_建ぺい率（％）_grpby_市区町村名,agg_count_容積率（％）_grpby_市区町村名,agg_mean_最寄駅：距離（分）_grpby_市区町村名,agg_mean_面積（㎡）_grpby_市区町村名,agg_mean_建ぺい率（％）_grpby_市区町村名,agg_mean_容積率（％）_grpby_市区町村名,agg_min_最寄駅：距離（分）_grpby_市区町村名,agg_min_面積（㎡）_grpby_市区町村名,agg_min_建ぺい率（％）_grpby_市区町村名,agg_min_容積率（％）_grpby_市区町村名,agg_max_最寄駅：距離（分）_grpby_市区町村名,agg_max_面積（㎡）_grpby_市区町村名,agg_max_建ぺい率（％）_grpby_市区町村名,agg_max_容積率（％）_grpby_市区町村名
0,11.0,55,1992.0,60.0,200.0,42,6.934821,4,0,0,0,0,0,0,0,0,0,0,-1,110.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0
1,45.0,80,1999.0,60.0,200.0,40,6.936086,4,0,0,1,0,1,0,0,0,1,1,-1,160.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0
2,6.0,30,1991.0,80.0,400.0,44,6.935683,3,0,0,2,1,2,1,0,0,2,0,-1,120.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0
3,19.0,20,1990.0,80.0,400.0,37,6.937919,5,0,0,3,2,2,1,0,1,2,-1,-1,80.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0
4,45.0,70,1975.0,60.0,200.0,37,6.937919,5,0,1,4,3,3,1,0,2,1,-1,-1,140.0,783,785,772,772,13.961686,63.229299,70.725389,334.481865,0.0,15,40.0,80.0,120.0,740,80.0,600.0
5,5.0,70,2008.0,80.0,400.0,10,7.031987,12,0,2,5,4,0,0,0,-1,2,0,-1,280.0,40,40,40,40,4.875000,65.125000,80.000000,395.000000,3.0,45,80.0,200.0,8.0,95,80.0,400.0
6,45.0,50,1990.0,80.0,400.0,16,7.022314,10,0,0,6,2,1,1,0,-1,2,0,0,200.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0
7,14.0,75,2010.0,60.0,200.0,19,6.996970,10,0,0,7,0,1,0,0,-1,1,-1,-1,150.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0
8,11.0,20,1990.0,80.0,600.0,46,6.930415,3,0,1,8,3,4,1,0,1,2,0,-1,120.0,783,785,772,772,13.961686,63.229299,70.725389,334.481865,0.0,15,40.0,80.0,120.0,740,80.0,600.0
9,17.0,70,2006.0,80.0,200.0,30,6.956425,7,0,0,7,0,1,0,0,-1,3,0,-1,140.0,633,633,629,629,16.146919,52.093207,70.413355,296.025437,2.0,15,60.0,200.0,45.0,95,80.0,600.0


# 新しいセクション